In [1]:
#importing the needed libraries
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_validate, KFold
import numpy as np
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, make_scorer, recall_score, f1_score, confusion_matrix, accuracy_score

In [2]:
#reading in the data we will use in the machine learning pipeline, with the top ten features of the method
df = pd.read_excel('pro_accgyro_training_set_10_final.xlsx')
df

,AccZ_Spectral_Variance,AccZ_RMS,AccZ_Spectral_Skewness,AccZ_Spectral_Mean,AccZ_Spectral_Kurtosis,AccZ_Spectral_Flatness,AccZ_Spectral_Entropy,AccZ_Spectral_Crest,AccZ_ZCR,AccZ_Spectral_Rolloff_50,Task_Label,Participant,Classification
0,0.009647,0.001076,0.638555,0.160822,0.348008,0.344127,13.584974,0.000274,0.527733,272.957495,12.0a,10785,0
1,0.009625,0.001083,0.568576,0.162448,0.094564,0.345113,13.588027,0.000262,0.525300,271.757481,12.0b,10785,0
2,0.009874,0.001092,0.607381,0.163459,0.144821,0.345735,13.588096,0.000239,0.524629,271.898173,12.0c,10785,0
3,0.010136,0.001098,0.649654,0.164157,0.329661,0.340929,13.582662,0.000272,0.528676,272.618181,12.0d,10785,0
4,0.009663,0.001078,0.652239,0.161347,0.356853,0.346463,13.588352,0.000289,0.522279,270.532640,12.0e,10785,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2701,0.012422,0.001175,0.744622,0.178976,0.806176,0.328023,13.575815,0.000298,0.516833,265.476033,85,94527,0
2702,0.113360,0.002589,3.133800,0.329863,13.855738,0.196810,13.331840,0.000672,0.429472,214.338917,87,94527,1
2703,0.027212,0.001323,2.498372,0.202672,11.005391,0.312232,13.469382,0.000559,0.506670,263.117386,89,94527,0
2704,0.060207,0.002128,2.645358,0.300402,14.623167,0.243584,13.466717,0.000794,0.453911,238.066075,91,94527,1


In [3]:
# extracting the features from the DataFrame
features = df.drop(columns=['Classification', 'Participant', 'Task_Label'])

# extracting the Classification (0/1) and participants from the dataframe
target = df['Classification'].values
participants = df['Participant'].values
task = df['Task_Label'].values

# extracting the feature names
feature_names = features.columns.tolist()

# converting the features, target, participants to numpy arrays
features_array = features.values
target_array = target

# creating the dictionary with the needed format to enter into the nested cross-validation
data_dict = {
    'data': features_array,
    'target': target_array,
    'feature_names': feature_names,
    'participants': participants,
    'task': task
}

data_dict

{'data': array([[9.64670000e-03, 1.07550000e-03, 6.38554700e-01, ...,
         2.74400000e-04, 5.27732700e-01, 2.72957495e+02],
        [9.62510000e-03, 1.08310000e-03, 5.68575600e-01, ...,
         2.62400000e-04, 5.25299600e-01, 2.71757481e+02],
        [9.87380000e-03, 1.09150000e-03, 6.07381000e-01, ...,
         2.39100000e-04, 5.24629200e-01, 2.71898173e+02],
        ...,
        [2.72120000e-02, 1.32320000e-03, 2.49837230e+00, ...,
         5.58800000e-04, 5.06670400e-01, 2.63117386e+02],
        [6.02072000e-02, 2.12800000e-03, 2.64535850e+00, ...,
         7.94300000e-04, 4.53911200e-01, 2.38066075e+02],
        [1.43461000e-02, 1.19270000e-03, 1.18200380e+00, ...,
         3.93000000e-04, 5.17238800e-01, 2.65831899e+02]]),
 'target': array([0, 0, 0, ..., 0, 1, 0]),
 'feature_names': ['AccZ_Spectral_Variance',
  'AccZ_RMS',
  'AccZ_Spectral_Skewness',
  'AccZ_Spectral_Mean',
  'AccZ_Spectral_Kurtosis',
  'AccZ_Spectral_Flatness',
  'AccZ_Spectral_Entropy',
  'AccZ_Spectral_Cre

In [5]:
#running the nested cross-validation for the Accelerometer method using Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score

# random seed for reproducibility
random_seed = 1
np.random.seed(random_seed)

X = data_dict['data']
Y = data_dict['target']
groups = data_dict['participants']

# setting up the parameter grid: the combinations of these will be tried out for ever outer loop's inner loop
param_grid = {
    'penalty' : ['l1', 'l2'],
    'C' : (1, 4, 20),
    'solver' : ['liblinear','lbfgs']
}


lrc = LogisticRegression(random_state=random_seed)

logo = LeaveOneGroupOut()


# defining the techniques to use for the inner and outer loops
inner_cv = KFold(n_splits=5, shuffle=True, random_state=1)
outer_cv = logo.split(X, Y, groups)

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='weighted'),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1_score': make_scorer(f1_score, average='macro'),
    'roc_auc': make_scorer(roc_auc_score, needs_proba=True, average='weighted')
}

# performing nested cross-validation
clf = GridSearchCV(estimator=lrc, param_grid=param_grid, cv=inner_cv)
cv_dic = cross_validate(clf, X, Y, cv=outer_cv, scoring=scoring, return_estimator=True, return_train_score=False)
mean_acc_score = cv_dic['test_accuracy'].mean()
mean_prec_score = cv_dic['test_precision'].mean()
mean_rec_score = cv_dic['test_recall'].mean()
mean_f1_score = cv_dic['test_f1_score'].mean()
mean_roc_auc = cv_dic['test_roc_auc'].mean()  # Mean ROC AUC score


print('nested_accuracy_scores:   ', cv_dic['test_accuracy'])
print('mean score:            {0:.4f}'.format(mean_acc_score))

print('nested_precision_scores:   ', cv_dic['test_precision'])
print('mean score:            {0:.4f}'.format(mean_prec_score))

print('nested_recall_scores:   ', cv_dic['test_recall'])
print('mean score:            {0:.4f}'.format(mean_rec_score))

print('nested_f1_scores:   ', cv_dic['test_f1_score'])
print('mean score:            {0:.4f}'.format(mean_f1_score))

print('nested_roc_auc_scores:   ', cv_dic['test_roc_auc'])
print('mean score:            {0:.4f}'.format(mean_roc_auc))

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-lear

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-lear

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-lear

...

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

nested_accuracy_scores:    [0.90909091 0.87878788 0.98484848 0.96969697 1.         1.
 0.96969697 1.         0.96969697 0.98484848 0.90909091 1.
 0.89393939 1.         1.         1.         0.93939394 0.84848485
 1.         1.         1.         0.74242424 0.96969697 1.
 0.95454545 1.         1.         1.         0.75757576 1.
 1.         1.         1.         0.96969697 0.92424242 1.
 1.         0.72727273 0.74242424 0.89393939 0.98484848]
mean score:            0.9494
nested_precision_scores:    [0.92045455 0.89818182 0.98520085 0.97107438 1.         1.
 0.96969697 1.         0.97107438 0.98520085 0.92045455 1.
 0.90909091 1.         1.         1.         0.94213287 0.87762238
 1.         1.         1.         0.81664099 0.97107438 1.
 0.95757576 1.         1.         1.         0.85454545 1.
 1.         1.         1.         0.97202797 0.93230174 1.
 1.         0.80909091 0.74798786 0.90909091 0.98545455]
mean score:            0.9587
nested_recall_scores:    [0.90909091 0.87878788

In [6]:
sensitivity_scores = []
specificity_scores = []

# iteration over the outer folds to calculate average sensitivity and specificity
for i, (train_index, test_index) in enumerate(logo.split(X, Y, groups)):
    print(f"Fold {i + 1}:")

    # getting the trained estimator for the present fold
    estimator = cv_dic['estimator'][i]
    
    # predicting on the test set
    y_pred = estimator.predict(X[test_index])
    
    # creating confusion matrix
    tn, fp, fn, tp = confusion_matrix(Y[test_index], y_pred).ravel()
    
    # calculating the sensitivity and specificity
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    print(f"Sensitivity: {sensitivity:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print("=" * 30)
    
    # store the sensitivity and specificity scores
    sensitivity_scores.append(sensitivity)
    specificity_scores.append(specificity)

    
    # calculate and print the averages of the sensitivity and specificity scores previously calculated
average_sensitivity = np.mean(sensitivity_scores)
average_specificity = np.mean(specificity_scores)

print("Average Sensitivity:", average_sensitivity)
print("Average Specificity:", average_specificity)

Fold 1:
Sensitivity: 0.7500
Specificity: 1.0000
Fold 2:
Sensitivity: 0.6667
Specificity: 1.0000
Fold 3:
Sensitivity: 0.9583
Specificity: 1.0000
Fold 4:
Sensitivity: 0.9167
Specificity: 1.0000
Fold 5:
Sensitivity: 1.0000
Specificity: 1.0000
Fold 6:
Sensitivity: 1.0000
Specificity: 1.0000
Fold 7:
Sensitivity: 0.9583
Specificity: 0.9762
Fold 8:
Sensitivity: 1.0000
Specificity: 1.0000
Fold 9:
Sensitivity: 0.9167
Specificity: 1.0000
Fold 10:
Sensitivity: 0.9583
Specificity: 1.0000
Fold 11:
Sensitivity: 0.7500
Specificity: 1.0000
Fold 12:
Sensitivity: 1.0000
Specificity: 1.0000
Fold 13:
Sensitivity: 0.7083
Specificity: 1.0000
Fold 14:
Sensitivity: 1.0000
Specificity: 1.0000
Fold 15:
Sensitivity: 1.0000
Specificity: 1.0000
Fold 16:
Sensitivity: 1.0000
Specificity: 1.0000
Fold 17:
Sensitivity: 0.9583
Specificity: 0.9286
Fold 18:
Sensitivity: 0.5833
Specificity: 1.0000
Fold 19:
Sensitivity: 1.0000
Specificity: 1.0000
Fold 20:
Sensitivity: 1.0000
Specificity: 1.0000
Fold 21:
Sensitivity: 1.0000


In [7]:
#print all performance scores that might be of interest
import scipy.stats

def calculate_stats(data):
    mean = np.mean(data)
    sd = np.std(data, ddof=1)  # using sample standard deviation (N-1)
    ci = scipy.stats.norm.interval(0.95, loc=mean, scale=sd/np.sqrt(len(data)))
    min_val = np.min(data)
    max_val = np.max(data)
    return mean, sd, ci, min_val, max_val

keys = ['test_accuracy', 'test_precision', 'test_recall', 'test_f1_score', 'test_roc_auc', 'sensitivity_scores', 'specificity_scores']

for key in keys:
    if key in cv_dic:
        data = cv_dic[key]
    elif key == 'sensitivity_scores':
        data = sensitivity_scores
    elif key == 'specificity_scores':
        data = specificity_scores

    stats = calculate_stats(data)
    print(f"{key}: Mean={stats[0]}, SD={stats[1]}, 95% CI={stats[2]}, Min={stats[3]}, Max={stats[4]}")


test_accuracy: Mean=0.9493717664449371, SD=0.07971552930120843, 95% CI=(0.9249712455983269, 0.9737722872915474), Min=0.7272727272727273, Max=1.0
test_precision: Mean=0.9586823170826261, SD=0.062020103859722106, 95% CI=(0.9396982765775455, 0.9776663575877067), Min=0.7479878611954084, Max=1.0
test_recall: Mean=0.9493717664449371, SD=0.07971552930120843, 95% CI=(0.9249712455983269, 0.9737722872915474), Min=0.7272727272727273, Max=1.0
test_f1_score: Mean=0.9387960192533372, SD=0.10174328948537967, 95% CI=(0.9076529122846794, 0.9699391262219951), Min=0.611764705882353, Max=1.0
test_roc_auc: Mean=0.9693912117692606, SD=0.06512485441362434, 95% CI=(0.9494568228028687, 0.9893256007356525), Min=0.73015873015873, Max=1.0
sensitivity_scores: Mean=0.8871951219512195, SD=0.19681577646240314, 95% CI=(0.826950806965032, 0.9474394369374071), Min=0.25, Max=1.0
specificity_scores: Mean=0.9849012775842044, SD=0.06111123680797165, 95% CI=(0.9661954366926424, 1.0036071184757664), Min=0.6190476190476191, Ma

In [8]:
# IF this is the best-performing nested CV for the method: print best hyperparameters for each outer loop
for fold_idx, estimator in enumerate(cv_dic['estimator']):
    print(f'Fold {fold_idx + 1} - Best hyperparameters: {estimator.best_params_}')


Fold 1 - Best hyperparameters: {'C': 20, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 2 - Best hyperparameters: {'C': 20, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 3 - Best hyperparameters: {'C': 20, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 4 - Best hyperparameters: {'C': 20, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 5 - Best hyperparameters: {'C': 20, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 6 - Best hyperparameters: {'C': 20, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 7 - Best hyperparameters: {'C': 20, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 8 - Best hyperparameters: {'C': 20, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 9 - Best hyperparameters: {'C': 20, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 10 - Best hyperparameters: {'C': 20, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 11 - Best hyperparameters: {'C': 20, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 12 - Best hyperparameters: {'C': 4, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 13 - Best hyperparame

In [9]:
# IF this is the best-performing nested CV for the method: print mode of hyperparameters

from collections import defaultdict, Counter

# initialize a dictionary to contain all hyperparameters and their values across loops
hyperparameters = defaultdict(list)

# extracting hyperparameters of each outer loop
for estimator in cv_dic['estimator']:
    for param, value in estimator.best_params_.items():
        hyperparameters[param].append(value)

# count the most frequent value for each hyperparameter
for param, values in hyperparameters.items():
    most_common_value, count = Counter(values).most_common(1)[0]  # Get the most frequent value
    print(f'Hyperparameter: {param} - Most frequent value: {most_common_value} (Count: {count})')


Hyperparameter: C - Most frequent value: 20 (Count: 40)
Hyperparameter: penalty - Most frequent value: l1 (Count: 41)
Hyperparameter: solver - Most frequent value: liblinear (Count: 41)
